# Problem Set 2

## by Arianna Michelangelo, Tatiana Bakwenye, Andrew Bennett

### Double Machine Learning Concept
Isolate the causal relationship between the variable(s) of interest (X) and the outcome variable (Y) by using a second machine learning model to isolate noisy covariates (D). A partial linear regression between the two models  then is used to find the conditional expectation of Y given accurate estimations of the covariates (X) and taking into account noisy parameters (D). 

In [28]:
# Import libraries
import pandas as pd
import yaml
import io
import klib as kl
import numpy as np
from numpy.random import seed
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import random
import category_encoders as ce
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
import numpy as np
from sklearn.base import clone
import doubleml as dml
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from xgboost import XGBClassifier
import pickle
seed(1234)

# Own preprocessing


### Data loading and EDA

In [2]:
# Load datasets
rhc_df = pd.read_csv('rhc.csv')
with open('confounders.yml', 'r') as file:
    confounders = yaml.safe_load(file)


In [3]:
rhc_df.shape

(5735, 63)

In [4]:
rhc_df.describe()

,Unnamed: 0,sadmdte,dschdte,dthdte,lstctdte,cardiohx,chfhx,dementhx,psychhx,chrpulhx,...,bili1,crea1,sod1,pot1,paco21,ph1,wtkilo1,adld3p,urin1,ptid
count,5735.000000,5735.000000,5734.000000,3722.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,...,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,1439.000000,2707.000000,5735.000000
mean,2868.000000,11638.686312,11660.050401,11753.869156,11781.257890,0.176635,0.178030,0.098344,0.067306,0.189887,...,2.267067,2.133017,136.768963,4.066693,38.748975,7.388413,67.827817,1.182071,2192.453665,5134.006452
std,1655.696228,513.967751,513.447322,538.812330,524.094168,0.381393,0.382571,0.297805,0.250573,0.392246,...,4.801538,2.053080,7.655160,1.028353,13.183445,0.109812,29.055534,1.819057,1525.140006,2972.206379
min,1.000000,10754.000000,10757.000000,10757.000000,10756.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.099991,0.099991,101.000000,1.099854,1.000000,6.579102,0.000000,0.000000,0.000000,5.000000
25%,1434.500000,11163.500000,11184.000000,11267.000000,11316.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.799927,1.000000,132.000000,3.399902,31.000000,7.339844,56.299990,0.000000,1110.000000,2561.500000
50%,2868.000000,11759.000000,11777.000000,11831.500000,11868.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.009766,1.500000,136.000000,3.799805,37.000000,7.399998,70.000000,0.000000,1927.000000,5131.000000
75%,4301.500000,12097.000000,12120.000000,12208.000000,12244.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.399902,2.399902,142.000000,4.599609,42.000000,7.459961,83.699950,2.000000,2955.000000,7689.000000
max,5735.000000,12441.000000,12560.000000,12783.000000,12644.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,58.195312,25.097656,178.000000,11.898438,156.000000,7.769531,244.000000,7.000000,9000.000000,10278.000000


In [5]:
rhc_df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


### Categorical treatment

In [6]:
rhc_df['ortho'] = rhc_df['ortho'].map({'No': 0, 'Yes': 1})
rhc_df['trauma'] = rhc_df['trauma'].map({'No': 0, 'Yes': 1})
rhc_df['seps'] = rhc_df['seps'].map({'No': 0, 'Yes': 1})
rhc_df['hema'] = rhc_df['hema'].map({'No': 0, 'Yes': 1})
rhc_df['meta'] = rhc_df['meta'].map({'No': 0, 'Yes': 1})
rhc_df['renal'] = rhc_df['renal'].map({'No': 0, 'Yes': 1})
rhc_df['gastr'] = rhc_df['gastr'].map({'No': 0, 'Yes': 1})
rhc_df['neuro'] = rhc_df['neuro'].map({'No': 0, 'Yes': 1})
rhc_df['card'] = rhc_df['card'].map({'No': 0, 'Yes': 1})
rhc_df['resp'] = rhc_df['resp'].map({'No': 0, 'Yes': 1})
rhc_df['dth30'] = rhc_df['dth30'].map({'No': 0, 'Yes': 1})
rhc_df['ca'] = rhc_df['ca'].map({'No': 0, 'Yes': 1})
rhc_df['sex'] = rhc_df['sex'].map({'Male': 0, 'Female': 1})
rhc_df['death'] = rhc_df['death'].map({'No': 0, 'Yes': 1})
rhc_df['swang1'] = rhc_df['swang1'].map({'No RHC': 0, 'RHC': 1})
rhc_df['dnr1'] = rhc_df['dnr1'].map({'No': 0, 'Yes': 1})


### Verify unique counts of cat cols

In [7]:
# Display unique values of categorical variables

# Define the function
def calculate_unique_counts(dataframe, columns):
    unique_counts = {}
    for column in columns:
        unique_values_count = dataframe[column].value_counts()
        unique_counts[column] = {'unique_count': dataframe[column].nunique(), 'value_counts': unique_values_count}
        # Print the results
        print(f"Number of unique values in {column}: {unique_counts[column]['unique_count']}")
        print(f"Value counts for {column}:")
        print(unique_values_count)
    return unique_counts

# List of columns for which we want to calculate unique counts
columns_to_check = ['cat1', 'cat2', 'race', 'income', 'ninsclas']

# Calculate unique counts for the specified columns
unique_counts = calculate_unique_counts(rhc_df, columns_to_check)

Number of unique values in cat1: 9
Value counts for cat1:
cat1
ARF                  2490
MOSF w/Sepsis        1227
COPD                  457
CHF                   456
Coma                  436
MOSF w/Malignancy     399
Cirrhosis             224
Lung Cancer            39
Colon Cancer            7
Name: count, dtype: int64
Number of unique values in cat2: 6
Value counts for cat2:
cat2
MOSF w/Sepsis        826
MOSF w/Malignancy    229
Coma                  90
Cirrhosis             38
Lung Cancer           15
Colon Cancer           2
Name: count, dtype: int64
Number of unique values in race: 3
Value counts for race:
race
white    4460
black     920
other     355
Name: count, dtype: int64
Number of unique values in income: 4
Value counts for income:
income
Under $11k    3226
$11-$25k      1165
$25-$50k       893
> $50k         451
Name: count, dtype: int64
Number of unique values in ninsclas: 6
Value counts for ninsclas:
ninsclas
Private                1698
Medicare               1458
Priva

### Encode Categoricals

In [8]:
# Initialize LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to the specified column and update the original dataframe
rhc_df['income'] = le.fit_transform(rhc_df['income'])

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


### Target Encoding

In [9]:
# Initialize TargetEncoder with smoothing
encoder = ce.TargetEncoder(cols=['cat1', 'cat2'], smoothing=10)

# Fit and transform the encoder on the specified columns with the target variable
rhc_df[['cat1', 'cat2']] = encoder.fit_transform(rhc_df[['cat1', 'cat2']], rhc_df['death'])

### One Hot Encoding

In [10]:
# Create dummy variables for 'race' and 'ninsclas' columns
race_dummies = pd.get_dummies(rhc_df['race'], prefix='race')
ninsclas_dummies = pd.get_dummies(rhc_df['ninsclas'], prefix='ninsclas')

# Concatenate the dummy variables with the original dataframe
rhc_df = pd.concat([rhc_df, race_dummies, ninsclas_dummies], axis=1)

# Drop the original 'race' and 'ninsclas columns if needed
rhc_df.drop(['race', 'ninsclas'], axis=1, inplace=True)


### Missing Data Treatment

In [11]:
# Identify features with missing values
features_with_missing_values = rhc_df.columns[rhc_df.isnull().any()].tolist()

# Display the count of missing values for each feature
missing_values_count = rhc_df[features_with_missing_values].isnull().sum()

print("Features with missing values:")

print(features_with_missing_values)

print("\nCount of missing values for each feature:")
print(missing_values_count)

Features with missing values:
['ca', 'dschdte', 'dthdte', 'adld3p', 'urin1']

Count of missing values for each feature:
ca          384
dschdte       1
dthdte     2013
adld3p     4296
urin1      3028
dtype: int64


In [12]:
# Drop columns since they have too many NaN
columns_to_exclude = ['cat2', 'adld3p', 'Unnamed: 0']
rhc_numerical = rhc_df.drop(columns=columns_to_exclude)


### Impute Values

In [13]:

# Impute NaN with KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputer.fit(rhc_numerical)
imputed_data = imputer.fit_transform(rhc_numerical)

imputed_df = pd.DataFrame(imputed_data, columns=rhc_numerical.columns)


/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a fu

In [14]:
# Check of it worked
features_with_missing_values = imputed_df.columns[imputed_df.isnull().any()].tolist()
missing_values_count = imputed_df[features_with_missing_values].isnull().sum()
print("Features with missing values:")
print(features_with_missing_values)
print("\nCount of missing values for each feature:")
print(missing_values_count)

Features with missing values:
[]

Count of missing values for each feature:
Series([], dtype: float64)


### Standardization

In [15]:
# Initialize StandardScaler and fit it to imputed_df
scaler = StandardScaler().fit(imputed_df)

# Transform imputed_df using the fitted scaler
imputed_df_scaled = scaler.transform(imputed_df)

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a fu

### Prepare Data

In [16]:
# Remove 'ninsclas' and 'race' from the list of confounders
confounders = [col for col in confounders if col not in ['ninsclas', 'race']]

# Add the specified columns to the list of confounders
confounders.extend(['race_black', 'race_other', 'race_white',
                    'ninsclas_Medicaid', 'ninsclas_Medicare', 'ninsclas_Medicare & Medicaid',
                    'ninsclas_No insurance', 'ninsclas_Private', 'ninsclas_Private & Medicare'])

In [17]:
# Prepare the data
Y = imputed_df['death']
D = imputed_df['swang1']
X = imputed_df[confounders]

# Models

### Random Forest with GridSearch

In [108]:
param_grid_classifier = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}


def optimize_and_cross_fit(model, X, y, param_grid):
    # Initialize GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=2, scoring='accuracy', refit=True, n_jobs=-1)
    
    # Fit GridSearchCV to find the best model
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    
    # Now perform your cross-validation with the best model
    kf = KFold(n_splits=2, shuffle=True, random_state=42)
    residuals = np.zeros(y.shape)
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        
        # Use the best model from GridSearchCV
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        residuals[test_index] = y.iloc[test_index] - y_pred
    
    return residuals, best_model


# Models 1
model_yc = RandomForestClassifier(random_state=42)
model_dc = RandomForestClassifier(random_state=42)

# Parameter grid for model optimization
param_grid_yc = param_grid_classifier # Define parameter grid for Y model
param_grid_dc = param_grid_classifier # Define parameter grid for D model

# Optimize models and calculate residuals
Y_residualsc, optimized_model_yc = optimize_and_cross_fit(model_yc, X, Y, param_grid_yc)
D_residualsc, optimized_model_dc = optimize_and_cross_fit(model_dc, X, D, param_grid_dc)

# Add a constant to the D residuals for the intercept in OLS
D_res_with_constc = sm.add_constant(D_residualsc)

# Linear regression
model_rf = sm.OLS(Y_residualsc, D_res_with_constc)
results_rf = model_rf.fit()

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: Fut

### XGBoost Model

In [110]:
param_grid_classifier = {
    'eta': [0.1, 0.2],  # Number of trees in the forest
    'max_depth': [3, 5, 10],  # Maximum depth of the tree
    'min_child_weight': [2, 5, 10],  # Minimum number of samples required to split an internal node
}


def optimize_and_cross_fit(model, X, y, param_grid):
    # Initialize GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=2, scoring='accuracy', refit=True)
    
    # Fit GridSearchCV to find the best model
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    
    # Now perform your cross-validation with the best model
    kf = KFold(n_splits=2, shuffle=True, random_state=42)
    residuals = np.zeros(y.shape)
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        
        # Use the best model from GridSearchCV
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        residuals[test_index] = y.iloc[test_index] - y_pred
    
    return residuals, best_model


# Models 1
model_yc = XGBClassifier(random_state=42)
model_dc = XGBClassifier(random_state=42)

# Parameter grid for model optimization
param_grid_yc = param_grid_classifier # Define parameter grid for Y model
param_grid_dc = param_grid_classifier # Define parameter grid for D model

# Optimize models and calculate residuals
Y_residualsc, optimized_model_yc = optimize_and_cross_fit(model_yc, X, Y, param_grid_yc)
D_residualsc, optimized_model_dc = optimize_and_cross_fit(model_dc, X, D, param_grid_dc)


# Add a constant to the D residuals for the intercept in OLS
D_res_with_constc = sm.add_constant(D_residualsc)

# Linear regression
model_xgboost = sm.OLS(Y_residualsc, D_res_with_constc)
results_xgboost = model_xgboost.fit()

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: Fut

### Professors Model

In [114]:
rhc = pd.read_csv("rhc.csv")
with open("confounders.yml", "r") as f:
    confounders = yaml.safe_load(f)

rhc["swang1"] = (rhc["swang1"] == "RHC").astype(int)
rhc["death"] = (rhc["death"] == "Yes").astype(int)
rhc.groupby("swang1")["death"].mean()

rhc_numerical = pd.get_dummies(rhc[['swang1', 'death'] + confounders], dtype=float, drop_first=True)


In [115]:
confounders_ = [c for c in rhc_numerical.columns if c not in ["swang1", "death"]]

# Prepare the data
Y = rhc_numerical['death']
D = rhc_numerical['swang1']
X = rhc_numerical[confounders_]

In [116]:
# DoubleMLData and DoubleMLPLR initialization
confounders_ = [c for c in rhc_numerical.columns if c not in ["swang1", "death"]]
data_dml = dml.DoubleMLData(
    rhc_numerical, y_col="death", d_cols="swang1", x_cols=confounders_
)

ml_m = GradientBoostingClassifier()
ml_l = GradientBoostingRegressor()
dml_plr_obj = dml.DoubleMLPLR(data_dml, ml_l, ml_m)

# Hyperparameter setting
learning_rate = [0.001, 0.01, 0.1]
n_estimators = [50, 100, 200]
max_depth = [1, 2, 3]

par_grids = {
    "ml_l": {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
    "ml_m": {
        "learning_rate": learning_rate,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
}

# Fine-tuning and effect estimation
dml_plr_obj.tune(param_grids=par_grids, tune_on_folds=True, search_mode="grid_search")

# read models
#dml_plr_obj = pickle.load(open('models/dml_plr_obj.pkl', 'rb'))

results = dml_plr_obj.fit()
results.summary

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: Fut

,coef,std err,t,P>|t|,2.5 %,97.5 %
swang1,0.039449,0.013358,2.953074,0.003146,0.013266,0.065631


In [117]:
D_residuals = D - np.array(dml_plr_obj.predictions['ml_m']).flatten()
Y_residuals = Y - np.array(dml_plr_obj.predictions['ml_l']).flatten() 
D_res_with_constc = sm.add_constant(D_residuals)

results_reg = sm.OLS(Y_residuals, D_res_with_constc).fit()

## Analysis

create dataframes for each model

In [119]:
rf_data = pd.read_csv(io.StringIO(results_rf.summary().tables[1].as_csv()), delimiter=',')
xgboost_data = pd.read_csv(io.StringIO(results_xgboost.summary().tables[1].as_csv()), delimiter=',')
rf_reg_data = pd.read_csv(io.StringIO(results_rf_reg.summary().tables[1].as_csv()), delimiter=',')
dlplr_data = pd.read_csv(io.StringIO(results_reg.summary().tables[1].as_csv()), delimiter=',')

rf_data = kl.clean_column_names(rf_data)
xgboost_data = kl.clean_column_names(xgboost_data)
rf_reg_data = kl.clean_column_names(rf_reg_data)
dlplr_data = kl.clean_column_names(dlplr_data)

create a dataframe of the results

In [134]:
cols = ['2xRF', '2xXGBoost', 'MLPLR']
coefs = [rf_data.loc[1,'coef'], xgboost_data.loc[1,'coef'],  dlplr_data.loc[1,'coef']]
se = [rf_data.loc[1,'std_err'], xgboost_data.loc[1,'std_err'],  dlplr_data.loc[1,'std_err']]
t_stats = [rf_data.loc[1,'t'], xgboost_data.loc[1,'t'], dlplr_data.loc[1,'t']]
p_vals = [rf_data.loc[1,'p_larger_|t|'], xgboost_data.loc[1,'p_larger_|t|'],  dlplr_data.loc[1,'p_larger_|t|']]
r_squared = [results_rf.rsquared, results_xgboost.rsquared,  results_reg.rsquared]
f_stats = [results_rf.fvalue, results_xgboost.fvalue, results_reg.fvalue]

# make a df
df = pd.DataFrame([coefs, se, t_stats, p_vals, r_squared, f_stats], columns=cols, index=['swang_coef', 'std_err', 't', 'p_vals', 'r**2', 'f_stats'])
df = df.T

##### Plot results for analysis

In [135]:
# plot the results in a stacked bar chart with plotly interactive plot

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='swang_coef', x=cols, y=df['swang_coef'])
])
fig.update_layout(title_text='swang coefficient')
fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})
fig.show()

the swang coefficient is the highest for the MLPLR model, followed by the 2xRF model, and then the 2xXGBoost model 

this indicates that in the MLPLR model the treatment effect is more sensitive to treatment assignment or in other words it is more sensitive to confounders than the other models.

the xgboost has the lowest swang coefficient, meaning treatment effect is the least sensitive. This means that the xgboost model is better at fitting without the use of the coefficient. 

In [136]:

fig = go.Figure(data=[
    go.Bar(name='std_err', x=cols, y=df['std_err']),
])
fig.update_layout(title_text='std errors')
fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})
fig.show()

the standard errors are very similar across the models, but highest in the MLPLR model. This means that the MLPLR is the least precise of the models

In [137]:

fig = go.Figure(data=[
    go.Bar(name='t', x=cols, y=df['t']),
])
fig.update_layout(title_text='t statistics')
fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})
fig.show()

the t stat is the highest in MLPLR, this indicates that the treatment effect is significant in the MLPLR model 

The higher the t stat, the more likely the coefficient is significant. 

In [138]:

fig = go.Figure(data=[
    go.Bar(name='p_vals', x=cols, y=df['p_vals']),
])
fig.update_layout(title_text='p values')
fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})
fig.show()

The p value is also the lowest in the MLPLR model, which also indicates that the treatment effect is significant. 

In [139]:

fig = go.Figure(data=[
    go.Bar(name='r_squared', x=cols, y=df['r**2']), 
])
fig.update_layout(title_text='r squared')
fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})
fig.show()

 the r squared variable is the highest for the MLPLR model, which means that the model explains the most variance in the outcome variable.

In [140]:

fig = go.Figure(data=[
    go.Bar(name='f_stats', x=cols, y=df['f_stats'])
])
fig.update_layout(title_text='f statistics')
fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})
fig.show()

the f statistic is highest for the MLPLR model which also indicates a better fit